In [6]:
from fairseq import tasks, utils, checkpoint_utils
import torch
from argparse import Namespace


In [7]:
args = Namespace(**{
    'task':'audio_finetuning',
    'data':'/home/harveen/fair/vakyansh-wav2vec2-experimentation/checkpoints/finetuning',
    'criterion':'ctc'
})
task = tasks.setup_task(args)

In [8]:
model, cfg, task = checkpoint_utils.load_model_ensemble_and_task(['/home/harveen/fair/vakyansh-wav2vec2-experimentation/checkpoints/finetuning/him_modified.pt'], task=task,
                                                                         arg_overrides={"w2v_path": "/home/harveen/fair/vakyansh-wav2vec2-experimentation/checkpoints/finetuning/clsril_modified.pt"})
model = model[0]
model.eval()

torch.save(model, '/home/harveen/fair/vakyansh-wav2vec2-experimentation/checkpoints/single_model/test.pt')

2022-05-24 10:58:48 | INFO | fairseq.models.wav2vec.wav2vec2_asr | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 1, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': '/home/harveen.chadha/vakyansh-wav2vec2-experimentation/logs/pretraining/tensorboard_2021-05-26_12-40-52', 'wandb_project': 'EKSTEP-PRETRAINING', 'azureml_logging': False, 'seed': 2021, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_

PicklingError: Can't pickle <enum 'Choices'>: attribute lookup Choices on fairseq.dataclass.constants failed

In [23]:
import torchaudio

In [25]:
arrr, _ = torchaudio.load('../evaluations/taarini_without_numbers/004-M-23_001.wav')

In [26]:
arrr

tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0003, -0.0003, -0.0003]])

In [28]:
logits = model(source=arrr, padding_mask=None)["encoder_out"]

In [30]:
predicted_ids = torch.argmax(logits[:, 0], dim=-1)

In [34]:
predicted_ids

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 52,  0, 64,  0,  0,  0,
         0,  0, 30,  0,  0,  0,  0,  0,  0,  0, 47,  0,  0,  4,  4,  0,  0,  0,
        46,  0,  0, 63,  0,  0, 45,  0, 47,  0,  0,  0,  4,  0, 52,  0,  0, 61,
         0,  0, 46,  0,  0, 55,  0,  0, 30,  0,  0,  0, 61,  0,  0,  0,  0, 27,
         0,  0,  0,  0,  4, 20,  0, 61,  0,  0,  0,  0,  4,  0, 25,  0,  0,  0,
         0, 56,  0,  0,  0,  0,  0, 27,  0,  0,  0,  4,  0, 20,  0, 61,  0,  0,
         4,  0,  0,  0, 20,  0,  0,  4,  0,  0, 42,  0,  0,  0, 52,  0, 57,  0,
         0, 35,  0,  0,  4,  4,  0,  0,  0,  8,  0,  0, 25,  0,  0, 66,  0, 26,
         0,  0,  0,  0,  0, 54,  0,  0,  0,  0,  0,  4,  0, 52,  0,  0, 62,  0,
         0,  0,  0,  0,  4,  0,  0, 20,  0, 66, 66,  0, 45,  0, 54,  0,  0,  0,
         0,  4,  0,  0,  0,  0,  0,  0, 

In [36]:
dict_ = []
with open('/home/harveen/fair/vakyansh-wav2vec2-experimentation/checkpoints/finetuning/dict.ltr.txt') as file:
    dict_ = file.readlines()

In [39]:
chars = [item.split(' ')[0] for item in dict_]

In [44]:

json_dict = {"<s>": 0, "<pad>": 1, "</s>": 2, "<unk>": 3}
for index, i in enumerate(chars):
    json_dict[i] = index + 4

In [45]:
import numpy as np
from itertools import groupby

class Decoder:
    def __init__(self, json_dict):
        self.dict = json_dict
        self.look_up = np.asarray(list(self.dict.keys()))

    def decode(self, ids):
        converted_tokens = self.look_up[ids]
        fused_tokens = [tok[0] for tok in groupby(converted_tokens)]
        output = ' '.join(''.join(''.join(fused_tokens).split("<s>")).split("|"))
        return output

In [46]:
decoder = Decoder(json_dict=json_dict)
print("Prediction: ", decoder.decode(predicted_ids))

Prediction:  होटल रॉयल हेरिटेज के चीज के क बहुत अच्छा है क्या 


In [47]:
look_up = np.asarray(list(json_dict.keys()))

In [48]:
look_up

array(['<s>', '<pad>', '</s>', '<unk>', '|', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ',
       'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ',
       'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ',
       'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श',
       'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै',
       'ॉ', 'ो', 'ौ', '्'], dtype='<U5')

In [51]:
converted_tokens = look_up[predicted_ids]
converted_tokens

array(['<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>',
       '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>',
       '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>',
       '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>',
       '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>',
       '<s>', '<s>', '<s>', 'ह', '<s>', 'ो', '<s>', '<s>', '<s>', '<s>',
       '<s>', 'ट', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', 'ल',
       '<s>', '<s>', '|', '|', '<s>', '<s>', '<s>', 'र', '<s>', '<s>',
       'ॉ', '<s>', '<s>', 'य', '<s>', 'ल', '<s>', '<s>', '<s>', '|',
       '<s>', 'ह', '<s>', '<s>', 'े', '<s>', '<s>', 'र', '<s>', '<s>',
       'ि', '<s>', '<s>', 'ट', '<s>', '<s>', '<s>', 'े', '<s>', '<s>',
       '<s>', '<s>', 'ज', '<s>', '<s>', '<s>', '<s>', '|', 'क', '<s>',
       'े', '<s>', '<s>', '<s>', '<s>', '|', '<s>', 'च', '<s>', '<s>',
       '<s>', '<s>', 'ी', '<s>', '<s>', '<s>', '<s>', '<s>', 'ज', '<s>',
       

In [59]:
ft = [tok[0] for tok in groupby(converted_tokens)]
' '.join(''.join(''.join(ft).split("<s>")).split('|'))

'होटल रॉयल हेरिटेज के चीज के क बहुत अच्छा है क्या '